## Introduction

Broccoli Project is a reward program in South Africa that helps the poor find shelters, buy food and clothes, etc. by giving them vouchers placed in “Broccoli Points” which are earned by performing socially beneficial actions such as:

  - Cleaning up community common grounds <br>
  - Taking an HIV/AIDS test<br>
  - Recycling <br>
  - Keeping their children in schools, etc. <br>
  
This case study represents the current situation in Cape Town, South Africa. 
The main objective of implementing broccoli points is to address the maximum number of people for the lowest number of Broccoli Points. There are three types of locations: districts, clinics, and redemption points. 

We will aim to locate Broccoli Point which are: <br>
- “near” to people‟s houses, so that people can reach those points <br>
- “near” to clinics so that people can go and take a test in case they do not have any points gathered when they need a food voucher <br>
- “near” to redemption points so that people can go and redeem the printed food vouchers. <br>

To this end, the three distances of interest are: <br>
1. District-to-Clinics (How far from home to where I can get healthcare) <br>
2. Clinics-to-Redemption (How far after I get care can I get food) <br>
3. District-to-Redemption (How far from my home to get food) 

## Project Goals

A) Try to cover all districts while minimizing the number of Broccoli points 
required to do so. Where do you place your BP’s? <br>
B) The decision-makers want to see how coverage changes based on the 
number of installed Broccoli points. Make a systematic sensitivity 
analysis to demonstrate how many people can be covered with fewer 
Broccoli points than the number you found in part A. <br>
C) Try to minimize the maximum distance between the districts & the 
Broccoli points while covering all districts. 

## Assumptions

- Broccoli points can  be implemented at any of the given geographical locations such as all the given locations of districts, clinics, and redemption points. 
- Since we used the given locations, all of the candidate locations can be maintained there are no obstacles to reaching the locations.
- Since candidate locations include districts, clinics, and redemption points those locations are inside the cellular radius naturally.
- Distance matrices’s data collection is explained above and the distance matrix can be reached by the Excel file. All missing distances are assumed according to a Python library.
- The coverage radius for α,β,and δ is equal and 7 km.


## Sets

$N$: Number of possible Broccoli Point (BP) locations = 60<br>
$I$: Number of districts = 15<br>
$J$: Number of clinics = 12 <br>
$K$: Number of redemption points = 33<br>

## Parameters

$P_{total}$ : Total population of 15 districts = $7,640,584$ <br>
$P_i$: Population of district i <br>
$D_{ni}$: Distance from district i to n <br>
$D_{nj}$: Distance from clinic j to n <br>
$D_{nk}$: Distance from redemption point k to n <br>
$\alpha$: Coverage radius for districts <br>
$\beta$: Coverage radius for clinics <br>
$\theta$: Coverage radius for redemption points <br>
\begin{equation}
   a_{ni}=\begin{cases}
    1, & \text{if $D_{ni} \le \alpha$}.\\
    0, & \text{otherwise}.
  \end{cases}
\end{equation}
\begin{equation}
   b_{nj}=\begin{cases}
    1, & \text{if $D_{nj} \le \beta$}.\\
    0, & \text{otherwise}.
  \end{cases}
\end{equation}
\begin{equation}
   c_{nk}=\begin{cases}
    1, & \text{if $D_{nk} \le \theta$}.\\
    0, & \text{otherwise}.
  \end{cases}
\end{equation}






## Decision Variables

$ z :$  maximum distance z from any chosen facility location to any district

\begin{equation}
  y_{i}=\begin{cases}
    1, & \text{if district i is served }.\\
    0, & \text{otherwise}.
  \end{cases}
\end{equation}

\begin{equation}
   x_n= \begin{cases}
    1, & \text{if Broccoli Point is built in location n}.\\
    0, & \text{otherwise}.
  \end{cases}
\end{equation}

 # Part A

### Objective Function

Try to cover all districts while minimizing the number of Broccoli points required to do so. <br> Where do you place your BP’s?

\begin{equation}
\begin{aligned}
& \min & \sum_{n=1}^{60} x_n\\
\end{aligned}
\end{equation}

### Constraints

1. Each district should be covered by at least one BP<br>

 $ \sum_{n=1}^{60} a_{ni} x_n \ge 1 \text , \forall i\in \text{{1,2,...,15}}$

2. Each BP should be in radius of at least one housing district

 $ \sum_{i=1}^{15} a_{ni} x_n \ge x_n \text , \forall n\in \text{{1,2,...,60}} $

3. Each BP should be in radius of at least one clinic

 $ \sum_{j=1}^{12} b_{nj} x_n \ge x_n \text , \forall n\in \text{{1,2,...,60}} $

4. Each BP should be in radius of at least one redemption point

 $ \sum_{k=1}^{33} c_{nk} x_n \ge x_n \text , \forall n\in \text{{1,2,...,60}} $

5. Domain constraint

$x_n \in \text{{0,1}}, \forall n\in \text{{1,2,...,60}}$

### Script for Part A

In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt

In [2]:
# extracting data
districts_df = pd.read_excel('candidate_locations_deneme.xlsx', sheet_name='Districts', usecols='F:T', skiprows=4)
clinics_df = pd.read_excel('candidate_locations_deneme.xlsx', sheet_name='Clinics', usecols='F:Q', skiprows=4)
redemptions_df = pd.read_excel('candidate_locations_deneme.xlsx', sheet_name='Redemptions', usecols='F:Q', skiprows=4)

districts_df = districts_df.apply(pd.to_numeric, errors='coerce')
clinics_df = clinics_df.apply(pd.to_numeric, errors='coerce')
redemptions_df = redemptions_df.apply(pd.to_numeric, errors='coerce')

# parameters
coverage_radius = 7
N = 60 
I = len(districts_df.columns)  
J = len(clinics_df.columns)    
K = len(redemptions_df.columns)  

# binary coverage parameters
a_ni = [[1 if districts_df.iloc[n, i] <= coverage_radius else 0 for n in range(N)] for i in range(I)]
b_nj = [[1 if clinics_df.iloc[n, j] <= coverage_radius else 0 for n in range(N)] for j in range(J)]
c_nk = [[1 if redemptions_df.iloc[n, k] <= coverage_radius else 0 for n in range(N)] for k in range(K)]# setting up the optimization model

# defining the model
m = gp.Model("Broccoli_Points_Placement")

# decision variable
x = m.addVars(N, vtype=GRB.BINARY, name="ifBPBuiltOnI") 

# objective function
m.setObjective(x.sum(), GRB.MINIMIZE)

# constraints
for i in range(I): 
    m.addConstr(gp.quicksum(a_ni[i][n] * x[n] for n in range(N)) >= 1, name=f"dist_cover_{i}")

for i in range(I): 
    for n in range(N):
        m.addConstr(gp.quicksum(a_ni[i][n] * x[n] for n in range(N)) >= x[n], name=f"clinic_cover_{i}")

for j in range(J):
    for n in range(N):
        m.addConstr(gp.quicksum(b_nj[j][n] * x[n] for n in range(N)) >= x[n], name=f"clinic_cover_{j}")

for k in range(K):
    for n in range(N):
        m.addConstr(gp.quicksum(c_nk[k][n] * x[n] for n in range(N)) >= x[n], name=f"redemp_cover_{k}")


# solving the model
m.optimize()

# output
if m.status == GRB.OPTIMAL:
    selected_bps = [n for n in range(N) if x[n].X > 0.5]
    print('Optimal number of BPs:', m.objVal)
    print('Selected BPs at locations:', selected_bps)

else:
    print("No feasible solution found.")


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-13
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13650HX, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2355 rows, 60 columns and 14148 nonzeros
Model fingerprint: 0xe3c79993
Variable types: 0 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 15.0000000
Presolve removed 2355 rows and 60 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.02 work units)
Thread count was 1 (of 20 available processors)

Solution count 2: 14 15 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.400000000000e+01, best 

### Analysis & Results for Part A

Model finds the minimum number of Broccoli Points initiated when cover radius is 7 km as 14. This means there can be at least 14 Broccoli Points initiated if all districts are desired to be covered for given coverage radius 7 km.

# Part B

The decision-makers want to see how coverage changes based on the 
number of installed <br> Broccoli points.  Make a systematic sensitivity 
analysis to demonstrate how many people <br> can be  covered with fewer 
Broccoli points than the number you found in part A. <br>

### Objective Function

\begin{equation}
\begin{aligned}
& \max & \sum_{i=1}^{15} y_{i}P_i\\
\end{aligned}
\end{equation}

### Constraints

1. Decreasing the value of BP found in Part A by one to do sensitivity analysis

$\sum_{n=1}^{60}x_n \le 13$

2. Each BP should be in radius of at least one housing district

 $ \sum_{j=1}^{12} b_{nj} x_n \ge x_n \text , \forall n\in \text{{1,2,...,60}} $

3. Each BP should be in radius of at least one redemption point

 $ \sum_{k=1}^{33} c_{nk} x_n \ge x_n \text , \forall n\in \text{{1,2,...,60}} $

4. Domain constraints

$x_n \text , y_{i} \in \text{{0,1}}, \forall n\in \text{{1,2,...,60}}, \forall i\in \text{{1,2,...,15}}$

### Script for Part B

In [3]:
# cleaning the model from Part A
m.reset()

Discarded solution information


In [4]:
# extracting data
districts_df = pd.read_excel('candidate_locations_deneme.xlsx', sheet_name='Districts', usecols='F:T', skiprows=4)
clinics_df = pd.read_excel('candidate_locations_deneme.xlsx', sheet_name='Clinics', usecols='F:Q', skiprows=4)
redemptions_df = pd.read_excel('candidate_locations_deneme.xlsx', sheet_name='Redemptions', usecols='F:Q', skiprows=4)

districts_df = districts_df.apply(pd.to_numeric, errors='coerce')
clinics_df = clinics_df.apply(pd.to_numeric, errors='coerce')
redemptions_df = redemptions_df.apply(pd.to_numeric, errors='coerce')

# parameters
coverage_radius = 7
N = 60  
I = districts_df.shape[1] 
J = clinics_df.shape[1]   
K = redemptions_df.shape[1]
a_ni = [[1 if districts_df.iloc[n, i] <= coverage_radius else 0 for i in range(I)] for n in range(N)]
b_nj = [[1 if clinics_df.iloc[n, j] <= coverage_radius else 0 for j in range(J)] for n in range(N)]
c_nk = [[1 if redemptions_df.iloc[n, k] <= coverage_radius else 0 for k in range(K)] for n in range(N)]
P = {
    1: 57996, 2: 52401, 3: 237414, 4: 391749, 5: 98468, 
    6: 3979, 7: 152030, 8: 200603, 9: 1261463, 10: 4977833, 
    11: 46684, 12: 52274, 13: 77476, 14: 26914, 15: 3300
}
P_total = 7640584


# defining the model
m = gp.Model("Broccoli_Points_Placement")

# decision variables
x = m.addVars(N, vtype=GRB.BINARY, name="if_BP_built_on_n")
y = m.addVars(I, vtype=GRB.BINARY, name="if_district_i_covered") 

# objective function
m.setObjective(gp.quicksum(y[i] * P[i+1] for i in range(I)), GRB.MAXIMIZE)

# constraints
m.addConstr(x.sum() <= 13, "limit_of_5_BPs") 

for i in range(I):
    m.addConstr(gp.quicksum(a_ni[n][i] * x[n] for n in range(N)) >= y[i], f"cover_district_{i}")

for j in range(J):
    for n in range(N):
        m.addConstr(gp.quicksum(b_nj[n][j] * x[n] for n in range(N)) >= x[n], f"cover_clinic_{j}")

for k in range(K):
    for n in range(N):
        m.addConstr(gp.quicksum(c_nk[n][k] * x[n] for n in range(N)) >= x[n], f"cover_redemption_{k}")

# solving the model
m.optimize()

# output
if m.status == GRB.OPTIMAL:
    selected_bps = [n for n in range(N) if x[n].X > 0.5]
    covered_districts = [i for i in range(I) if y[i].X > 0.5]
    covered_population = sum(P[i+1] for i in covered_districts)
    print(f'Optimal covered population: {covered_population}')
    print(f'Percentage of total population covered: {covered_population / P_total * 100}%')
    print('Selected BPs at locations:', selected_bps)
else:
    print("No feasible solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13650HX, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 1456 rows, 75 columns and 7987 nonzeros
Model fingerprint: 0x2543bea2
Variable types: 0 continuous, 75 integer (75 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+03, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 617 rows and 4 columns
Presolve time: 0.01s
Presolved: 839 rows, 71 columns, 4642 nonzeros
Variable types: 0 continuous, 71 integer (71 binary)

Root relaxation: objective 7.640584e+06, 41 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7640584.00    

### Analysis & Results for Part B

Sensitivity analysis is done according to following algorithm. Optimal number of broccoli points found in part A for specific cover radius. Population percantage that is served by broccoli points is 100 in Part A as broccoli points are expected to serve everybody. Let’s say there is a budget and budget is exceeded when there are 14 broccoli points. Therefore, percantage of population covered when there are less broccoli points initiated should be analyzed. Analysis will done when there are 13, 12, 11 broccoli points. If percantage of population that gets the service from broccoli points when there are less broccoli point than in part A is above the certain threshold that satisfies, less broccoli points will be initiated.

Population percantage covered with 14 BP when the coverage radius is 7km : 100

Population percantage covered with 13 BP when the coverage radius is 7km: 99.24

Population percantage covered with 12 BP when the coverage radius is 7km: 98.23

Population percantage covered with 11 BP when the coverage radius is 7km: 86.64

Let’s assume the treshold is covering at least 95 percent of population. Then 12 is the minimum number of broccoli points. Also, it is obvious that decrease in covered percentage will level up as less broccoli points are inititated because when maximizing the population covered; first candidate broccoli points are initiated for the most populated districts and when the number of broccoli points are decreased model will try to keep the most populous ones and relieve the least populous ones. Therefore, at first model relieve the Wetton, the least populated one with 3000 people to maximize the population covered and did not serve the Wetton. The model will keep the Cape Town until the minimum number of Broccoli points for feasible solution for given coverage radius because Cape Town is the most populous one

# Part C

Try to minimize the maximum distance between the districts & the 
Broccoli points while covering all districts. 

### Objective Function

\begin{equation}
     \begin{aligned}
         \min  z \\
    \end{aligned}
\end{equation}

### Constraints

1. Each district should be covered by at least one BP<br>

 $ \sum_{n=1}^{60} a_{ni} x_n \ge 1 \text , \forall i\in \text{{1,2,...,15}}$

2. Maximum distance z from any chosen facility location to any district is minimized

$x_n D_{ni} \le z \text , \forall n\in \text{{1,2,...,60}}\text , \forall i\in \text{{1,2,...,15}}$

3. Each BP should be in radius of at least one housing district

 $ \sum_{i=1}^{15} a_{ni} x_n \ge x_n \text , \forall n\in \text{{1,2,...,60}} $

4. Each BP should be in radius of at least one clinic

 $ \sum_{j=1}^{12} b_{nj} x_n \ge x_n \text , \forall n\in \text{{1,2,...,60}} $

5. Each BP should be in radius of at least one redemption point

 $ \sum_{k=1}^{33} c_{nk} x_n \ge x_n \text , \forall n\in \text{{1,2,...,60}} $

6. Domain canstraints

$x_n \in \text{{0,1}}, \forall n\in \text{{1,2,...,60}}$

### Script for Part C

In [5]:
# cleaning the model from Part B
m.reset()

Discarded solution information


In [6]:
# extracting data
districts_df = pd.read_excel('candidate_locations_deneme.xlsx', sheet_name='Districts', usecols='F:T', skiprows=4)
clinics_df = pd.read_excel('candidate_locations_deneme.xlsx', sheet_name='Clinics', usecols='F:Q', skiprows=4)
redemptions_df = pd.read_excel('candidate_locations_deneme.xlsx', sheet_name='Redemptions', usecols='F:Q', skiprows=4)

# data to numeric
districts_df = districts_df.apply(pd.to_numeric, errors='coerce')
clinics_df = clinics_df.apply(pd.to_numeric, errors='coerce')
redemptions_df = redemptions_df.apply(pd.to_numeric, errors='coerce')

# parameters
coverage_radius = 7
N = 60
I = len(districts_df.columns)  
J = len(clinics_df.columns)    
K = len(redemptions_df.columns)  

# binary coverage parameters
a_ni = [[1 if districts_df.iloc[n, i] <= coverage_radius else 0 for n in range(N)] for i in range(I)]
b_nj = [[1 if clinics_df.iloc[n, j] <= coverage_radius else 0 for n in range(N)] for j in range(J)]
c_nk = [[1 if redemptions_df.iloc[n, k] <= coverage_radius else 0 for n in range(N)] for k in range(K)]

# defining the model
m = gp.Model("Max_Distance")

# decision variable
x = m.addVars(N, vtype=GRB.BINARY, name="ifBPBuiltOnI") 
z = m.addVar(vtype=GRB.INTEGER, name="max_distance")  

# objective function
m.setObjective(z, GRB.MINIMIZE)

# constraints
for i in range(I): 
    m.addConstr(gp.quicksum(a_ni[i][n] * x[n] for n in range(N)) >= 1, name=f"dist_cover_{i}")
    
for n in range(N):
    for i in range(I):
        m.addConstr(x[n] * districts_df.iloc[n, i] <= z, name=f"max_dist_{n}_{i}")

for i in range(I): 
    for n in range(N):
        m.addConstr(gp.quicksum(a_ni[i][n] * x[n] for n in range(N)) >= x[n], name=f"clinic_cover_{i}")

for j in range(J):
    for n in range(N):
        m.addConstr(gp.quicksum(b_nj[j][n] * x[n] for n in range(N)) >= x[n], name=f"clinic_cover_{j}")

for k in range(K):
    for n in range(N):
        m.addConstr(gp.quicksum(c_nk[k][n] * x[n] for n in range(N)) >= x[n], name=f"redemp_cover_{k}")


# solving the model
m.optimize()

# output
if m.status == GRB.OPTIMAL:
    print('max distance:', m.objVal)

else:
    print("No feasible solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13650HX, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 3255 rows, 61 columns and 15930 nonzeros
Model fingerprint: 0x73923754
Variable types: 0 continuous, 61 integer (60 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3255 rows and 61 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.03 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: 124 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.240000000000e+02, best bound 1.240000000000e+02, gap 0.0000%
max distance: 124.0


### Analysis & Results for Part C

Maximum distance between any district and initiated Broccoli Point is found to be 124 km by model when coverage radius is 7 km. To linearize the objective function of minimizing the maximum distance between any district and initiated Broccoli Points, new variable z is introduced. Z is set be the maximum distance between any district and initiated Broccoli Point by constraint number (1). Objective function is minimizing the z value. 